# PettingZoo Speaker-Listener Environment Demonstration

## Introduction

[PettingZoo](https://www.pettingzoo.ml/) is a Python library for conducting research in multi-agent reinforcement learning, akin to a multi-agent version of [Gym](https://github.com/openai/gym). It implements a variety of environments, including:
- [Atari](https://www.pettingzoo.ml/atari): Multi-player Atari 2600 games (cooperative, competitive and mixed sum)
- [Butterfly](https://www.pettingzoo.ml/butterfly): Cooperative graphical games developed by us, requiring a high degree of coordination
- [Classic](https://www.pettingzoo.ml/classic): Classical games including card games, board games, etc.
- [MAgent](https://www.pettingzoo.ml/magent): Configurable environments with massive numbers of particle agents, originally from https://github.com/geek-ai/MAgent
- [MPE](https://www.pettingzoo.ml/mpe): A set of simple nongraphical communication tasks, originally from https://github.com/openai/multiagent-particle-envs
- [SISL](https://www.pettingzoo.ml/sisl): 3 cooperative environments, originally from https://github.com/sisl/MADRL

<img src="https://www.pettingzoo.ml/mpe/mpe_simple_speaker_listener.gif" width="500" align="center"/>

The [Simple Speaker Listener Environment](https://www.pettingzoo.ml/mpe/simple_speaker_listener) is implemented in the MPE library. It is a 2-agent environment in which one agent, the "speaker", has information about the goal and has a limmited mode of communication with the second agent, the "listener", which must use the speaker's communications and its limitted observations to navigate a 2D space toward the goal. The speaker agent cannot navigate, and the listener object cannot communicate.

<img src="images/speaker_listener_screenshot.png" width="500" align="center"/>

In [1]:
!pip install 'pettingzoo[mpe]'

## Environment Description

The latest implementation of the speaker-listener environment is `simple_speaker_listener_v3`. We can create an environment instance using the `env` function which accepts two parameters:
1. `max_cycles` - the number of actions each agent can perform before the end of the episode. _default=25_
2. `continuous_actions` - if `True`, both the speaker and the listener have a continuous action space. otherwise they are discrete, finite spaces. _default=False_

The environment object implements many usful tools to help understand and properly utilize the environment. Below we use the `agents` attribute toiterate over the agent names, and the `observation_space` and `action_space` functions to show the [gym spaces](https://gym.openai.com/docs/#spaces) for the agents observation and action spaces.

In [2]:
from pettingzoo.mpe import simple_speaker_listener_v3
import numpy as np


def print_env_info(continuous_actions):
    env = simple_speaker_listener_v3.env(continuous_actions=continuous_actions)
    env.reset()
    
    print('continuous actions:' if continuous_actions else 'discrete actions:')
    
    for i, agent in enumerate(env.agents, 1):
        print(f'- agent {i}: {agent}')
        print(f'\t- observation space: {env.observation_space(agent)}')
        print(f'\t- action space: {env.action_space(agent)}')


print_env_info(continuous_actions=False)
print()
print_env_info(continuous_actions=True)

discrete actions:
- agent 1: speaker_0
	- observation space: Box(-inf, inf, (3,), float32)
	- action space: Discrete(3)
- agent 2: listener_0
	- observation space: Box(-inf, inf, (11,), float32)
	- action space: Discrete(5)

continuous actions:
- agent 1: speaker_0
	- observation space: Box(-inf, inf, (3,), float32)
	- action space: Box(0.0, 1.0, (3,), float32)
- agent 2: listener_0
	- observation space: Box(-inf, inf, (11,), float32)
	- action space: Box(0.0, 1.0, (5,), float32)


### Observation Spaces

The `Box(low, high, shape, dtype)` space contains any vector of shape `shape` that contains only values within the closed interval between `low` and `high` represented as type `dtype`. Both the speaker and the listener receive one dimensional `Box` observations of different sizes with any 32-bit floating point value. Note that the observation spaces remain the same regardless of the action space type (continuous / discrete). We can get the next acting agent's observation using the environment's `last` function, which returns the previous observation, reward, "done" flag, info dictionary. The current acting agent is chosen sequentially according to the agents' order in the `agents` attribute. The next agent is chosen when calling the `step` function which is sets the agent's action.

#### Speaker
The speaker observation is of type `Box(-inf, inf, (3,), float32)`, which is any vector of 3 dimensions. The values represent the RGB color of the goal to which the listener must navigate to maximize rewards.

#### Listener
The speaker observation is of type `Box(-inf, inf, (11,), float32)`, which is any vector of 11 dimensions. The first two values are the agent's velocity in 2D space. The next six values are the red, blue, and green landmarks' positions relative to the listener. The last three values correspond to communication received from the speaker. Below is a precise ordering of the values in the observation vector:
1. listener agent velocity X
2. listener agent velocity Y
3. red landmark X pos - listener agent X pos
4. red landmark Y pos - listener agent Y pos
5. blue landmark X pos - listener agent X pos
6. blue landmark Y pos - listener agent Y pos
7. green landmark X pos - listener agent X pos
8. green landmark Y pos - listener agent Y pos
9. communication channel 1
10. communication channel 2
11. communication channel 3

Note that the communication observation (values 9, 10, and 11) will always be 0 in the first round since no communication has yet been received from the speaker

In [3]:
env = simple_speaker_listener_v3.env()
env.reset()  # reset the environment, selected agent is "speaker_0"

# run twice to show the chnage in the communication vector
for i in range(2):
    #speaker obs
    obs, _, _, _ = env.last()  # get speaker observation vector
    print(f'agnet: {env.agents[0]}')
    print(f'observation: {obs}')
    print()
    env.step(0)  # send discrete message "A". next agent is selected (listener_0)
    
    obs, _, _, _ = env.last()  # get listener observation vector
    print(f'agnet: {env.agents[1]}')
    print(f'observation: {obs}')
    print()
    env.step(1)  # perform the "go left" action

agnet: speaker_0
observation: [0.65 0.15 0.15]

agnet: listener_0
observation: [ 0.          0.          0.35338476 -0.6476226  -0.10971016  0.06285673
 -0.4768763   1.0360808   0.          0.          0.        ]

agnet: speaker_0
observation: [0.65 0.15 0.15]

agnet: listener_0
observation: [-0.5         0.          0.40338477 -0.6476226  -0.05971016  0.06285673
 -0.42687628  1.0360808   1.          0.          0.        ]



### Action Spaces

The action agents' action spaces can be either discrete or continuous, depending on the `continuous_actions` parameter. If discrete, the action spaces are of type `Discrete(n)`, which contains the integer values 0 to n-1. Otherwise, the action spaces are of type `Box` (like the observation spaces), but with values constrained between 0 and 1. Actions are given sequentially by agent order according the `agents` attribute by using the `step` function. The given action must be one from the corresponding agent's action space.

#### Discrete Actions

##### Speaker
The action space is `Discrete(3)` containing the values 0, 1, 2. Each value corresponds to a possible message. Value 0 corresponds to message A, which can be seen in the communication vector of the listener's observation as \[1, 0, 0\]. Similarly, values 1 and 2 correspond to messages B and C and appear as \[0, 1, 0\] and \[0, 0, 1\] in the listener's observation respectively.

##### Listener
Discrete:  
The action space is `Discrete(5)` containing the values 0 - 4. Each value applies force on the agent, increasing its velocity to some direction. The velocity will slowly deteriorate until the agent stops, unless constant force is applied. The values' meanings are as follows:
* 0 - do nothing
* 1 - push left (add velocity in negative x-axis direction)
* 2 - push right (add velocity in positive x-axis direction)
* 3 - push down (add velocity in negative y-axis direction)
* 4 - push up (add velocity in positive t-axis direction)

In [4]:
# speaker action-to-index dict
SPEAKER_DISCRETE_ACTIONS = {
    'A': 0,
    'B': 1,
    'C': 2
}

# listener action-to-index dict
LISTENER_DISCRETE_ACTIONS = {
    'nothing': 0,
    'left':    1,
    'right':   2,
    'down':    3,
    'up':      4
}

env = simple_speaker_listener_v3.env(continuous_actions=False)  # discrete actions env
env.reset()

# CHANGE ACTION ACCORDING TO THE SPEADER TABLE AND SEE THE LISTENER'S COMMUNICATION OBSERVATIONS CHANGE
chosen_speaker_action = SPEAKER_DISCRETE_ACTIONS['A']

# CHANGE ACTION ACCORDING TO THE LISTENER TABLE AND SEE THE VELOCITY OBSERVATIONS CHANGE
chosen_listener_action = LISTENER_DISCRETE_ACTIONS['left']

# run twice to show the chnage in the communication vector
for i in range(2):
    #speaker action
    obs, _, _, _ = env.last()
    print(f'agnet: {env.agents[0]}')
    print(f'observation: {obs}')
    print()
    env.step(chosen_speaker_action)  
    
    # listener action
    obs, _, _, _ = env.last()
    print(f'agnet: {env.agents[1]}')
    print(f'observation: {obs}')
    print()
    env.step(chosen_listener_action)  

agnet: speaker_0
observation: [0.15 0.15 0.65]

agnet: listener_0
observation: [ 0.          0.         -0.8839424   0.15246797 -0.4558873   0.70290554
  0.7304127   0.18869944  0.          0.          0.        ]

agnet: speaker_0
observation: [0.15 0.15 0.65]

agnet: listener_0
observation: [-0.5         0.         -0.8339424   0.15246797 -0.4058873   0.70290554
  0.78041273  0.18869944  1.          0.          0.        ]



#### Continuous Actions

##### Speaker
The action space is `Box(0.0, 1.0, (3,), float32)` containing 3D vectors of values in \[0, 1\]. The values have no specific meaning, and are given, as is, as the listener observation's communication vector.

##### Listener
The action space is `Box(0.0, 1.0, (5,), float32)` containing 3D vectors of values in \[0, 1\]. Each value describes the amount of force applied in each direction (left, right, up, and down). Below is a precise ordering of the values in the action vector:
1. No force (useful for some implementations)
2. Force right (positive in x-axis)
3. Force left (negative in x-axis)
4. Force up (positive in y-axis)
5. Force down (negative in y-axis)

In [5]:
# speaker continuous action function
def speaker_continuous_action(v1, v2, v3):
    return np.array([v1, v2, v3], dtype=np.float32)

# listener continuous action function
def listener_continuous_action(right, left, up, down):
    return np.array([0, right, left, up, down], dtype=np.float32)

env = simple_speaker_listener_v3.env(continuous_actions=True)  # continuous actions env
env.reset()

# CHANGE ACTION AS NEEDED
chosen_speaker_action = speaker_continuous_action(v1=0.5, v2=1.0, v3=0.2)

# CHANGE ACTION AS NEEDED
chosen_listener_action = listener_continuous_action(right=0.8, left=0.8, up=0.5, down=0.7)

# run twice to show the chnage in the communication vector
for i in range(2):
    #speaker action
    obs, _, _, _ = env.last()
    print(f'agnet: {env.agents[0]}')
    print(f'observation: {obs}')
    print()
    env.step(chosen_speaker_action)  
    
    # listener action
    obs, _, _, _ = env.last()
    print(f'agnet: {env.agents[1]}')
    print(f'observation: {obs}')
    print()
    env.step(chosen_listener_action)  

agnet: speaker_0
observation: [0.65 0.15 0.15]

agnet: listener_0
observation: [ 0.          0.          0.25383052  0.36080134  0.567426   -0.21938929
 -0.28110778  0.171347    0.          0.          0.        ]

agnet: speaker_0
observation: [0.65 0.15 0.15]

agnet: listener_0
observation: [ 0.         -0.09999999  0.25383052  0.37080133  0.567426   -0.2093893
 -0.28110778  0.18134701  0.5         1.          0.2       ]



## Running the Environment

We can run a game simulation by resetting an environment and playing out the episode. This is done by iterating over the agents repeatedly, providing an action for each agent at every iteration, until we wish to stop or until the `max_cycles` limit has been reached. In the example below, we define a policy function to generate random actions within each agent's action space.

### Policies

We implement a policy as a function that, given the current observation, returns an action to perform. Below we define a policy class that supports both `Discrete` and `Box` type action spaces that completely ignores the observation and samples a random valid action from the given action space.

In [6]:
from gym.spaces import Discrete, Box

# define a random policy for continuous action agents.
# the policy returns a numpy array of the action space shape with random values between 0 and 1.
class RandomPolicy:
    def __init__(self, action_space):
        # choose a policy function for this action space type
        if isinstance(action_space, Discrete):  # discrete action policy
            self.policy_fn = self.__discrete_policy
        elif isinstance(action_space, Box):  # continuous action policy
            self.policy_fn = self.__continuous_policy
        else:  # other types are not supported
            raise TypeError(f'action_space must be of type Box or Discrete. got {type(action_space).__name__}')
        
        self.action_space = action_space
        
    def __call__(self, observation):
        # we completely ignore the observation and create a random valid action.
        return self.policy_fn()
    
    def __discrete_policy(self):
        # a random number within the discrete action range
        return np.random.randint(self.action_space.n)
    
    def __continuous_policy(self):
        # a random vector within the continuous range of the appropriate dimensionality
        # convert to the right dtype to avoid clipping warnings (e.g. float64 to float32)
        return np.random.uniform(self.action_space.low, self.action_space.high, self.action_space.shape).astype(self.action_space.dtype)

### Simulation

We now define an environment with either a discrete or continuous action space that limits to a small number of steps, for the purposes of this demonstration, using the `max_cycles` parameter. We then create a policy for each agent and start iterating over them repeatedly. For this, the environment implements the `agent_iter` function. This function simply iterates over the list of agents `max_cycles` times, allowing us to perform `max_cycles` steps for each agent. An added bonus of using `agent_iter` is that it raises an error if there was no `step` call within the iteration (which can prevent horrible bugs). After the episode has ended, the agents' "done" status will be true, and . In this case the episode is complete and the environment must be reset if we wish to run it again.

In [7]:
# CHOOSE MAX CYCLES AND DISCRETE OR CONTINUOUS ACTION SPACE
env = simple_speaker_listener_v3.env(max_cycles=5, continuous_actions=True)
env.reset()

# create a random policy for both agents' action spaces.
policies = {
    env.agents[0]: RandomPolicy(env.action_space(env.agents[0])),
    env.agents[1]: RandomPolicy(env.action_space(env.agents[1]))
}

# iterate over agents until the episode is complete
for agent in env.agent_iter():
    observation, reward, done, info = env.last()

    # if done, the episode is complete. no more actions can be taken
    if done:
        break
    
    # choose an action and execute
    action = policies[agent](observation)
    env.step(action)
    
    # log everything
    print(f'{agent} reward:      {reward}')
    print(f'{agent} observation: {observation}')
    print(f'{agent} action:      {action}')
    print()

speaker_0 reward:      0.0
speaker_0 observation: [0.65 0.15 0.15]
speaker_0 action:      [0.8003149  0.22646305 0.34666488]

listener_0 reward:      0.0
listener_0 observation: [ 0.          0.         -0.40749893  0.86787695  0.02217458  0.70224684
 -0.20109619 -0.2215645   0.          0.          0.        ]
listener_0 action:      [0.07327786 0.33796993 0.19610927 0.27309957 0.71906203]

speaker_0 reward:      -0.9642981758802109
speaker_0 observation: [0.65 0.15 0.15]
speaker_0 action:      [0.2752149 0.4685041 0.7870879]

listener_0 reward:      -0.9642981758802109
listener_0 observation: [ 0.07093033 -0.22298123 -0.41459197  0.8901751   0.01508154  0.72454494
 -0.20818922 -0.19926639  0.8003149   0.22646305  0.34666488]
listener_0 action:      [0.15150264 0.7362857  0.10083412 0.9906669  0.4533504 ]

speaker_0 reward:      -0.978476535079686
speaker_0 observation: [0.65 0.15 0.15]
speaker_0 action:      [0.46705744 0.2645737  0.26079756]

listener_0 reward:      -0.9784765350796

### Rendering

We can render the environment to see visualize the observation space. We must call the `render` function at every iteration to create and update a rendering in a separate window. Below we show a 100-step episode controlled by our random policies.

In [8]:
# long episode for interesting rendering.
env = simple_speaker_listener_v3.env(max_cycles=100, continuous_actions=True)
env.reset()

# create a random policy for both agents' action spaces.
policies = {
    env.agents[0]: RandomPolicy(env.action_space(env.agents[0])),
    env.agents[1]: RandomPolicy(env.action_space(env.agents[1]))
}

# run an episode
for agent in env.agent_iter():
    observation, reward, done, info = env.last()
    
    # stop if done
    if done:
        break
    
    # choose and execute action
    action = policies[agent](observation)
    env.step(action)
    
    # render the environment
    env.render('human')

# This line is SUPPOSED to close the rendering window, but it does not.
# restart the kernel close the window, and then don't run this cell again.
env.close()

## Advanced Tools

### Wrappers

PettingZoo provides utilities called wrappers. They are used to alter the behavior of the environment with minimal effort wihtout changing the general environment API. In fact, the Simple Speaker Listener environment already wrapped upon creation. In the below demonstration, we can see that the environment type is actually a wrapper called `OrderEnforcingWrapper`. This class works like the original environment and adds checks that enforce the agent order, and adds extra functionality, e.g. the `agent_iter` function is implemented in this wrapper (and is not available in the raw environment). The underlying environment, revealed using the environment's `env` attribute, is actually wrapped by another wrapper called `AssertOutOfBoundsWrapper`, which checks that given actions are compatible with the agents' discrete action space (similarly, the continuous action space uses another wrapper called `ClipOutOfBoundsWrapper`). Only under this wrapper do we find the raw environment object. However, we can jump directly to the raw environment by using the `unwrapped` property.

In [9]:
env = simple_speaker_listener_v3.env()
print(f'external wrapper:      {type(env)}')
print(f'inner wrapper:         {type(env.env)}')
print(f'raw environment:       {type(env.env.env)}')
print(f'environment unwrapped: {type(env.unwrapped)}')

external wrapper:      <class 'pettingzoo.utils.wrappers.order_enforcing.OrderEnforcingWrapper'>
inner wrapper:         <class 'pettingzoo.utils.wrappers.assert_out_of_bounds.AssertOutOfBoundsWrapper'>
raw environment:       <class 'pettingzoo.mpe.simple_speaker_listener_v3.raw_env'>
environment unwrapped: <class 'pettingzoo.mpe.simple_speaker_listener_v3.raw_env'>


#### Custom Wrappers

Using the `BaseWrapper` abstraction, we can create our own wrappers. We demonstrate this below with a custom wrapper that turns this environment into a single-agent environment. This is done by skipping the speaker's step before the user has a chance to do so. The action will be the goal color if continuous or some constant message for each color if discrete. We the user to  

In [10]:
from pettingzoo.utils import wrappers
import numpy as np

class ListenerOnlyWrapper(wrappers.BaseWrapper):
    
    def __init__(self, env):
        super().__init__(env)
        
        # reset to skip speaker before new game
        self.reset()
        
        # set single agent list
        self.agents = self.agents[1:]
        
    def reset(self):
        super().reset()
        
        # skip speaker action
        self.__step_speaker()
    
    def step(self, action):
        super().step(action)  # do listener action
        
        # skip speaker action
        self.__step_speaker()
        
    def __step_speaker(self):
        _, _, done, _ = self.env.last()
        goal_color, _, done, _ = self.env.last()
        
        # speaker is done before the listener.
        if done:
            return
        
        # step with the correct action type
        if self.env.unwrapped.continuous_actions:
            super().step(goal_color)
        else:
            super().step(np.argmax(goal_color))

We can add wrap a new environment by creating a new wrapper instance initialized with the wrapped environment.

In [11]:
env = simple_speaker_listener_v3.env(max_cycles=5, continuous_actions=True)
env = ListenerOnlyWrapper(env)
print(f'custom wrapped environment: {env}')
print(f'list of agents:             {env.agents}')

custom wrapped environment: ListenerOnlyWrapper<simple_speaker_listener_v3>
list of agents:             ['listener_0']


Like before, we can simulate an episode with `agent_iter` which will now always select the listener. Note that the initial observation contains a communication vector of 0 because no communication is received before the first step. We can see the listener's observation in the following observations until the end of the episode

In [12]:
policy = RandomPolicy(env.action_space(env.agents[0]))
for agent in env.agent_iter():
    observation, reward, done, info = env.last()
    
    # stop if done
    if done:
        break
    
    # choose and execute action
    action = policy(observation)
    env.step(action)
    
    # log everything
    print(f'{agent} reward:      {reward}')
    print(f'{agent} observation: {observation}')
    print(f'{agent} action:      {action}')
    print()

listener_0 reward:      0.0
listener_0 observation: [ 0.          0.          0.86115235  0.19098532  1.5698947  -0.10147929
  0.20799956 -0.43903372  0.          0.          0.        ]
listener_0 action:      [0.04389883 0.493977   0.1372025  0.03613445 0.84057933]

listener_0 reward:      -0.7646347726512953
listener_0 observation: [ 0.17838725 -0.40222245  0.84331363  0.23120756  1.552056   -0.06125704
  0.19016084 -0.39881146  0.65        0.15        0.15      ]
listener_0 action:      [0.13004364 0.91196555 0.7504862  0.16325119 0.9451058 ]

listener_0 reward:      -0.7657352388132257
listener_0 observation: [ 0.21453011 -0.6925942   0.8218606   0.30046698  1.5306029   0.00800237
  0.16870782 -0.32955205  0.65        0.15        0.15      ]
listener_0 action:      [0.3019261  0.8184272  0.8000573  0.69732434 0.8271661 ]

listener_0 reward:      -0.7765990997561083
listener_0 observation: [ 0.17008254 -0.5843665   0.80485237  0.35890362  1.5135946   0.06643902
  0.15169957 -0.2711

### Parallel Environments

Up until now we were able to view each agent's observations and act individually, even though the actions were only applied after a full cycle through all the agents. Many PettingZoo environments support acting in parallel using yet another wrapper, including Simple Speaker Listener. This is important for implementing algorithms that consider joint actions, e.g., centralized control. We can create a parallel environment by invoking the `parallel_env` function.

In [13]:
env = simple_speaker_listener_v3.parallel_env()

In this environment, both the observations of the speaker and listener agents are bundled together in a dictionary. The initial environment observations is returned from the `reset` function.

In [14]:
observations = env.reset()
observations

{'speaker_0': array([0.15, 0.65, 0.15], dtype=float32),
 'listener_0': array([ 0.        ,  0.        , -1.5332555 , -0.17393556, -1.3299489 ,
         0.10230795, -1.3239901 , -0.88061666,  0.        ,  0.        ,
         0.        ], dtype=float32)}

Actions are performed jointly using the `step` function. This will return the next observation bundle together with separate dictionaries for the reward and "done" status for each agent.

In [15]:
joint_action = {'speaker_0': 1, 'listener_0': 2}
observations, rewards, done, info = env.step(joint_action)

print('new observations:')
print(observations)
print('step rewards:')
print(rewards)
print('done status:')
print(done)

new observations:
{'speaker_0': array([0.15, 0.65, 0.15], dtype=float32), 'listener_0': array([ 0.5       ,  0.        , -1.5832555 , -0.17393556, -1.3799489 ,
        0.10230795, -1.37399   , -0.88061666,  0.        ,  1.        ,
        0.        ], dtype=float32)}
step rewards:
defaultdict(<class 'int'>, {'speaker_0': -1.9147259002215258, 'listener_0': -1.9147259002215258})
done status:
{'speaker_0': False, 'listener_0': False}


Putting it all together, a simulation might look something like this:

In [16]:
env = simple_speaker_listener_v3.parallel_env(max_cycles=5)
observations = env.reset()

# a random policy for each agent
policies = {
    env.agents[0]: RandomPolicy(env.action_space(env.agents[0])),
    env.agents[1]: RandomPolicy(env.action_space(env.agents[1]))
}

for _ in range(env.unwrapped.max_cycles):
    joint_action = {agent: policies[agent](obs) for agent, obs in observations.items()}
    observations, rewards, done, info = env.step(joint_action)
    
    if any(done.values()):
        break
    
    # log everything
    print(f'rewards:      {rewards}')
    print(f'observations: {observations}')
    print(f'actions:      {joint_action}')
    print()

rewards:      defaultdict(<class 'int'>, {'speaker_0': -1.4730577606576092, 'listener_0': -1.4730577606576092})
observations: {'speaker_0': array([0.15, 0.65, 0.15], dtype=float32), 'listener_0': array([ 0.        ,  0.        , -1.705754  ,  0.55679   , -0.7011504 ,
        0.9906795 ,  0.00769451,  0.30910408,  1.        ,  0.        ,
        0.        ], dtype=float32)}
actions:      {'speaker_0': 0, 'listener_0': 0}

rewards:      defaultdict(<class 'int'>, {'speaker_0': -1.4730577606576092, 'listener_0': -1.4730577606576092})
observations: {'speaker_0': array([0.15, 0.65, 0.15], dtype=float32), 'listener_0': array([ 0.        ,  0.        , -1.705754  ,  0.55679   , -0.7011504 ,
        0.9906795 ,  0.00769451,  0.30910408,  1.        ,  0.        ,
        0.        ], dtype=float32)}
actions:      {'speaker_0': 0, 'listener_0': 0}

rewards:      defaultdict(<class 'int'>, {'speaker_0': -1.3764898118077564, 'listener_0': -1.3764898118077564})
observations: {'speaker_0': array([0

### Other Environments

There already exists a single-agent and single-landmark version of Simple Speaker Listener, called Simple. Here, the one agent acts like the listener and has a simplified observation space containing the agent's velocity and the landmark's relative location. There are other, more complex environments in the MPE library and other environment libraries in PettingZoo. You can explore them in the [PettingZoo Website](https://www.pettingzoo.ml/envs).

In [17]:
from pettingzoo.mpe import simple_v2
import numpy as np


def print_env_info(continuous_actions):
    env = simple_v2.env(continuous_actions=continuous_actions)
    env.reset()
    
    print('continuous actions:' if continuous_actions else 'discrete actions:')
    
    for i, agent in enumerate(env.agents, 1):
        print(f'- agent {i}: {agent}')
        print(f'\t- observation space: {env.observation_space(agent)}')
        print(f'\t- action space: {env.action_space(agent)}')


print_env_info(continuous_actions=False)
print()
print_env_info(continuous_actions=True)

discrete actions:
- agent 1: agent_0
	- observation space: Box(-inf, inf, (4,), float32)
	- action space: Discrete(5)

continuous actions:
- agent 1: agent_0
	- observation space: Box(-inf, inf, (4,), float32)
	- action space: Box(0.0, 1.0, (5,), float32)
